***Project Title***

**AUTHORSHIP ATTRIBUTION USING AVERAGE WORD2VEC AND PREDICTION COMPARISON FOR DIFFERENT VECTOR DIMENSION**

---



***Submitted By***

Humaira Afia



***Objective***

---



In this work, the purpose is to predict the correct label of the text. For this, I have used a dataset containing Bangla text. Here texts are documented with 16 different tags.
  
***Step by Step Working Process***

---



*   I have prepared the text documents so that it can be usable for this work. I have splitted this dataset into 3 parts. one part is for Training, 2nd part is for Validation and another part is for Testing.

*   Data cleaning is a necessary thing for this work. Noisy and unnecessary data can reduce the performance. So why, removing english words, degits, special & unsupported characters, removing HTML tags, tokenizing, removing stop words, stemming, Feature extraction etcetera has been done in pre-processing steps.  

*   For feature extraction I have used Average Word2Vec of size 300 dimension and 3000 dimension.

*   Then, I have applied Support Vector Machine & Logistic Regression for 300 dimension numarical data and 3000 dimension numarical data to predict how accurately these model can classify the texts.

*   10-fold Cross Validation have been done to examine wheather model is overfitting or underfitting.

*  Lastly I have compared the performance using same algorithm with different vector size.










## **Import Necessary Libraries**

 Bellow the libraries are imported as needed for this project.

In [ ]:
%matplotlib inline
import nltk
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import sqlite3
import re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive

/gdrive/My Drive


In [ ]:
%cd /gdrive/My Drive/SM_FRIEND_NEWSDATA

/gdrive/My Drive/SM_FRIEND_NEWSDATA


## **Load The Data Corpus**

In [ ]:
#Read the text corpus
data = pd.read_csv('Data_Collection.csv').apply(lambda x: x.astype(str).str.lower())

#remove duplicate data
data.drop_duplicates(keep=False, inplace=True)

#show sample of the text corpus
data.head()

,label,text,is_valid
0,shunil_gongopaddhay,থেকে কত দূরে চলে এসেছে ভরত। সে হেসে উঠল আপন মন...,false
1,humayun_ahmed,এতে ভয় কমে যায়। বল একটা গল্প।’ ‘তুমি বল।’ আনিস...,false
2,shomresh,হবে। ওই দেখুন ওর এক চোখ কানা। ডান দিকটা দিয়ে দ...,false
3,humayun_ahmed,"বললাম, আপনি ওর গায়ে হাত দিলেন কেন? ষণ্ডাগণ্ডা ...",false
4,humayun_ahmed,হত! আবার চাদর মুড়ি দিয়ে নিজেকে গুটিয়ে ফেলি। যে...,false


## **Size Of The Corpus**

This corpus cotains 14374 text documents with 3 columns. I have worked with 'text' column and 'label' column.

In [ ]:
#shape of the text corpus
data.shape

(14374, 3)

## **Pre-Processing Steps**

In [ ]:
#Removing Stopwords
stopwords = pd.read_csv('Stopwords.csv')
def to_encode(bangla_word):
    return bangla_word.encode('utf-8')

def to_decode(bangla_word):
    return bangla_word.decode('utf-8')

set_stop = set(stopwords['words'].map(to_encode))


#Stemming

from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()

texts = data['text'].values


#Cleaning HTML tags

import re
def cleanhtml(sentence):
    cleanr1 = re.compile('/.*?')
    cleantext = re.sub(cleanr1,' ',sentence)
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

#Cleaning Punctuations, English Words, Mathematical digits   

def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    cleaned = re.sub(r'[A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z]', r'', cleaned1)
    cleaned1 = re.sub(r'[1|2|3|4|5|6|7|8|9|0]', r'', cleaned)
    return cleaned1 
    

# Making a function for pre-processing the corpus  

def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word1 in cleanhtml(w).split():
                for cleaned_word in clean_punct(cleaned_word1).split():
                    if len(cleaned_word)>2:
                        if(to_encode(cleaned_word) not in set_stop):
                            s = stemmer.stem_word(cleaned_word)
                            if len(s)>2:
                                final_words.append(s)
                                filtered_sentence.append(s)
                        else:
                            continue
                    else:
                        continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

##**Getting Clean Text and Label**

In [ ]:
#Getting possibly clean and noisefree data corpus
X = pre_process(data['text'].values)

#Getting the labels of the text documents
Y = data['label'].values

## **Splitting The Data corpus for Train, Test & Validation**
 
Here, I have taken 67% of the corpus for training, 33% of the corpus for testing and 33% of the corpus for validation. 

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle=Flase)# this is for time series split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33,random_state=42) # this is random splitting
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33,random_state=42) # this is random splitting

## **Feature Extraction**

As machine can't compute document data, it is needed to convert document into numarical data.For feature extraction here, I have used Average Word2Vec with 300 dimension giving 300 dimension vector value for each word.

In [ ]:
#split the sentence of training documents

i=0
list_of_sentance_train=[]
for sentance in X_train:
    list_of_sentance_train.append(sentance.split()) #split the sentence of training documents

In [ ]:
#import libraries ffrom gensim for making my own Wor2Vec model
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#training word2vec model on the given list of sentences in X_train
w2v_model=Word2Vec(list_of_sentance_train,min_count=5,size=300, workers=10)
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  32430
sample words  ['বাড়ি', 'এলাম', 'কথা', 'মোড়', 'তোমা', 'হ্যাঁ', 'গাড়ি', 'গেলাম', 'ভাল', 'লাবুর', 'হাতখানা', 'মুঠো', 'চেপ', 'চোখ', 'বোজ', 'প্রীতম', 'গাঢ়স্বর', 'কাকা', 'শোনো', 'শিলিগুড়ি', 'কষ্ট', 'আমি', 'যাব', 'স্কুল', 'কান্না', 'পাব', 'পরদিন', 'চেম্বার', 'ঢুক', 'বোস', 'সাহেব', 'হাসিমুখ', 'লাইক', 'ট্রিপ', 'নর্থ', 'বেঙ্গল', 'দীপনাথ', 'একটু', 'হকচকি', 'আনন্', 'জবাব', 'একদম', 'ট্যুর', 'থাক', 'দায়িত্ব', 'বাড়', 'পার', 'অফিস', 'ঢেল', 'সাজা']


**Convert Train Data text into 300 dimension vector**

In [ ]:
# computing average word2vec for each sentence in X_train

sent_vectors = []; # the avg-w2v for each sentence is stored in this list
for sent in tqdm(list_of_sentance_train): # for each sentence 
    sent_vec = np.zeros(300) # as I have used google's w2v, vector size is given 300
    cnt_words =0; # num of words with a valid vector in the sentence
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors_train = np.array(sent_vectors)

100%|██████████| 6452/6452 [14:24<00:00,  7.46it/s]


**Convert Validation Data text into 300 dimension vector**

In [ ]:
i=0
list_of_sentance_cv=[]
for sentance in X_cv:
    list_of_sentance_cv.append(sentance.split())

In [ ]:
# computing average word2vec for each sentence in X_cv

sent_vectors_cv = []; # the avg-w2v for each sentence
for sent in tqdm(list_of_sentance_cv): # for each sentence
    sent_vec = np.zeros(300) 
    cnt_words =0; # num of words with a valid vector in the sentence
    for word in sent: # for each word in a sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_cv.append(sent_vec)
sent_vectors_cv = np.array(sent_vectors_cv)

100%|██████████| 3178/3178 [08:27<00:00,  6.26it/s]


**Convert Test Data text into 300 dimension vector**

In [ ]:
i=0
list_of_sentance_test=[]
for sentance in X_test:
    list_of_sentance_test.append(sentance.split())

In [ ]:
# computing average word2vec for each sentence in X_test

sent_vectors_test = []; # the avg-w2v for each sentence
for sent in tqdm(list_of_sentance_test): # for each sentence
    sent_vec = np.zeros(300) 
    cnt_words =0; # num of words with a valid vector in the sentence
    for word in sent: # for each word in sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
sent_vectors_test = np.array(sent_vectors_test)

100%|██████████| 4744/4744 [11:14<00:00,  7.03it/s]


### **Applying Support Vector Machine and Logistic Regression**


**SVM classifier**

In [ ]:
#import necessary libraries

from sklearn . naive_bayes import GaussianNB
from sklearn import metrics
from sklearn . metrics import accuracy_score
from sklearn . model_selection import cross_val_score

In [ ]:
import sklearn
from sklearn import svm

smodel = svm . SVC ( C = 3, gamma = 'scale', kernel = 'rbf' )
smodel . fit ( sent_vectors , y_train )

SVC(C=3, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

**Cross validation score for svm**

In [ ]:
#using 10-fold
accuracy = cross_val_score(smodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.97 0.95 0.93 0.94 0.91 0.95 0.95 0.9  0.93 0.95]


In [ ]:
#get the mean of each fold 
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 93.7384679483364


**Training accuracy using SVM**

In [ ]:
y_train_pred  = smodel . predict ( sent_vectors )
print ( 'Training accuracy %s' % accuracy_score (y_train , y_train_pred ))

Training accuracy 0.9685368877867327


**Prediction score on Test Data**

In [ ]:
from sklearn . metrics import classification_report
y_test_pred  = smodel . predict ( sent_vectors_test )

In [ ]:
label=np.unique(y_train)

In [ ]:
print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred ) )

Model accuracy 0.9542580101180439
accuracy normalized 4527
                     precision    recall  f1-score   support

            bongkim       0.93      0.94      0.94       157
      humayun_ahmed       0.99      0.99      0.99      1194
manik_bandhopaddhay       0.87      0.84      0.85       136
                mzi       0.97      0.98      0.97       275
             nazrul       0.83      0.59      0.69        59
 nihar_ronjon_gupta       0.95      0.94      0.94       112
       robindronath       0.92      0.93      0.92       324
         shirshendu       0.96      0.93      0.95       285
           shomresh       0.99      0.98      0.98       350
          shordindu       0.96      0.95      0.95       203
      shorotchandra       0.92      0.94      0.93       366
      shottojit_roy       0.98      0.98      0.98       245
shunil_gongopaddhay       0.97      0.98      0.98       524
        tarashonkor       0.84      0.88      0.86       215
     toslima_nasrin      

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
Lmodel = LogisticRegression ( multi_class = 'ovr' , solver = 'lbfgs' )
Lmodel . fit ( sent_vectors , y_train )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Cross Validation Score for Logistic Regression**

In [ ]:
#using 10 fold cross validation

from sklearn import metrics
from sklearn . metrics import accuracy_score
from sklearn . model_selection import cross_val_score
accuracy = cross_val_score(Lmodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10)
print(accuracy)

[0.95 0.94 0.92 0.91 0.91 0.9  0.9  0.89 0.91 0.95]


In [ ]:
#get the mean of each fold 

print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100)

Accuracy of Model with Cross Validation is: 91.72519492887326


**Training Accuracy using Logistic Regression**

In [ ]:
#train accuracy
y_train_pred = Lmodel . predict(sent_vectors)
print ( 'Model accuracy %s' % accuracy_score (  y_train , y_train_pred ) )

Model accuracy 0.9460632362058277


**Prediction score on Test Data**

In [ ]:
from sklearn . metrics import classification_report
from sklearn . model_selection import cross_val_score

y_test_pred  = Lmodel . predict ( sent_vectors_test )

print ( 'Model accuracy %s' % accuracy_score ( y_test , y_test_pred ) )
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False ) )
print ( classification_report ( y_test , y_test_pred,target_names=label  ) )

Model accuracy 0.9395025295109612
accuracy normalized 4457
                     precision    recall  f1-score   support

            bongkim       0.91      0.94      0.92       157
      humayun_ahmed       0.98      0.99      0.98      1194
manik_bandhopaddhay       0.93      0.65      0.76       136
                mzi       0.96      0.95      0.96       275
             nazrul       0.87      0.44      0.58        59
 nihar_ronjon_gupta       0.91      0.88      0.89       112
       robindronath       0.84      0.94      0.89       324
         shirshendu       0.95      0.94      0.95       285
           shomresh       0.98      0.97      0.98       350
          shordindu       0.95      0.94      0.95       203
      shorotchandra       0.89      0.92      0.90       366
      shottojit_roy       0.96      0.99      0.98       245
shunil_gongopaddhay       0.97      0.99      0.98       524
        tarashonkor       0.83      0.82      0.83       215
     toslima_nasrin      

### **Taking 20 % as train and validation data of the Corpus**

Here, I have taken 80% of the corpus for training, 20% of the corpus for testing and 20% of the corpus for validation. And here, I have taken vector size as 3000 dimension

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42) # this is random splitting
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2,random_state=42) # this is random splitting

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
# this line of code trains your w2v model on the give list of sentances
w2v_model=Word2Vec(list_of_sentance_train,min_count=5,size=30, workers=10)
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words)

number of words that occured minimum 5 times  39945
sample words  ['করিলাম\u200c', 'বিভক্ত', 'ফাঁ', 'কুমীর', 'দাঁত', 'ভোঁতা', 'মুখ', 'ধারালো', 'মিলাইয়া', 'চেহারা', 'দুশ্চরিত্র', 'পতিভক্তি', 'রেবা', 'সীতা', 'সাবিত্রীর', 'সমতুল্য', 'না\u200c', 'সেজন্য', 'তাঁহা', 'দোষ', 'সুনীল', 'সরক', 'স্পষ্টতা', 'রাম', 'সত্যবান', 'সমকক্ষ', 'বোক', 'কিছুক্ষণ', 'দ্বার', 'দাঁড়াইয়া', 'রহিল\u200c', 'শেষ', 'ভাঙা', 'গলা', 'বলিল\u200c', 'ব্যোমকেশ', 'খবর', 'কাগজ', 'দৃষ্', 'নিবদ্ধ', 'বসিয়াছিল\u200c', 'ঘাড়', 'ফিরাইয়া', 'আসুন', 'ক্যাবল', 'ভাব', 'লইয়া', 'টেবিল', 'আসিয়া', 'দাঁড়াইল;', 'বলিব', 'গোবর', 'শুষ্ক', 'কঠিন', 'তাহা', 'আপাদমস্তক', 'নিরীক্ষণ', 'অভিন', 'ভাল', 'হচ্ছে\u200c', 'নির্বোধ', 'থাপ', 'চেয়ার', 'বসিয়া', 'পড়িল\u200c', 'চক্ষ', 'পরিদর্শন', 'স্বলিত', 'স্বর', 'কী—', 'বলছ', 'বোকামির', 'করব', 'আলোচনা', 'লাভ', 'পৈতৃক', 'সম্পত্তি', 'পাব', 'জন্য', 'মানুষ', 'খুন', 'করেছেন;', 'এক\u200c', 'হুকুম', 'সিং', 'নিয়ে', 'স্ত্রী', 'মের', 'ষড়যন্ত্র', 'সরানো', 'দরক', 'ছিল;', 'বেঁচ', 'থাকল', 'জীবন', 'আপনা', 'দোহন', 'করত', 'টিল', 'প

**Converting Train Data text into 3000 dimension vector**

In [ ]:
# Converting TrainDatatext to vector
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_train): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors_train = np.array(sent_vectors)

100%|██████████| 9199/9199 [27:32<00:00,  5.57it/s]


**Converting Validation Data text into 3000 dimension vector**

In [ ]:
i=0
list_of_sentance_cv=[]
for sentance in X_cv:
    list_of_sentance_cv.append(sentance.split())


# average Word2Vec for validation part

sent_vectors_cv = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_cv): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_cv.append(sent_vec)
sent_vectors_cv = np.array(sent_vectors_cv)
#print(np.asarray(sent_vectors_cv).shape)
#print(sent_vectors_cv[0])


100%|██████████| 2300/2300 [07:38<00:00,  5.01it/s]


**Converting Test Data text into 3000 dimension vector**

In [ ]:
i=0
list_of_sentance_test=[]
for sentance in X_test:
    list_of_sentance_test.append(sentance.split())

# average Word2Vec for test data text

sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_of_sentance_test): # for each review/sentence
    sent_vec = np.zeros(3000) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
sent_vectors_test = np.array(sent_vectors_test)
   

100%|██████████| 2875/2875 [09:18<00:00,  5.15it/s]


**Prediction using Support Vector Machine**

In [ ]:
smodel = svm . SVC ( C = 3, gamma = 'scale', kernel = 'rbf' )
smodel . fit ( sent_vectors , y_train )

#cross validation on validation data
accuracy = cross_val_score(smodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10) #10-fold cross validation
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100) #find validation accuracy

#training accuracy
y_train_pred  = smodel . predict (sent_vectors)
print ( 'Training accuracy %s' % accuracy_score (y_train , y_train_pred))

#prediction on test data
from sklearn . metrics import classification_report
y_test_pred  = smodel . predict ( sent_vectors_test )

#showing classification report
print ( 'Model accuracy %s' % accuracy_score (y_test , y_test_pred))
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False))
print ( classification_report ( y_test , y_test_pred))

Accuracy of Model with Cross Validation is: 95.3913043478261
Training accuracy 0.9855419067289923
Model accuracy 0.9812173913043478
accuracy normalized 2821
                     precision    recall  f1-score   support

            bongkim       0.96      1.00      0.98        89
      humayun_ahmed       1.00      0.99      0.99       738
manik_bandhopaddhay       0.96      0.98      0.97        83
                mzi       0.99      0.98      0.99       168
             nazrul       0.96      0.82      0.89        33
 nihar_ronjon_gupta       0.97      0.97      0.97        74
       robindronath       0.96      0.98      0.97       201
         shirshendu       0.99      0.97      0.98       177
           shomresh       1.00      1.00      1.00       204
          shordindu       0.98      0.97      0.98       111
      shorotchandra       0.97      0.96      0.97       231
      shottojit_roy       0.99      1.00      0.99       150
shunil_gongopaddhay       0.99      0.99      0.9

**Prediction using Logistic regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
Lmodel = LogisticRegression ( multi_class = 'ovr' , solver = 'lbfgs' )
Lmodel . fit ( sent_vectors , y_train )
#cross validation
accuracy = cross_val_score(smodel, sent_vectors_cv, y_cv, scoring='accuracy', cv = 10) #10-fold cross validation
print("Accuracy of Model with Cross Validation is:",accuracy.mean() * 100) #find validtion accuracy

#training accuracy
y_train_pred  = Lmodel . predict (sent_vectors)
print ( 'Training accuracy %s' % accuracy_score (y_train , y_train_pred))

#prediction on test data
from sklearn . metrics import classification_report
y_test_pred  = Lmodel . predict ( sent_vectors_test )

#showing classification report
print ( 'Model accuracy %s' % accuracy_score (y_test , y_test_pred))
print ( 'accuracy normalized %s' % accuracy_score ( y_test , y_test_pred , normalize = False))
print ( classification_report ( y_test , y_test_pred))

Accuracy of Model with Cross Validation is: 95.3913043478261
Training accuracy 0.9704315686487661
Model accuracy 0.9676521739130435
accuracy normalized 2782
                     precision    recall  f1-score   support

            bongkim       0.92      0.97      0.95        89
      humayun_ahmed       0.99      0.99      0.99       738
manik_bandhopaddhay       0.99      0.80      0.88        83
                mzi       0.98      0.98      0.98       168
             nazrul       0.95      0.61      0.74        33
 nihar_ronjon_gupta       0.97      0.96      0.97        74
       robindronath       0.90      0.97      0.93       201
         shirshendu       0.98      0.97      0.98       177
           shomresh       0.99      1.00      0.99       204
          shordindu       0.99      0.97      0.98       111
      shorotchandra       0.93      0.94      0.94       231
      shottojit_roy       0.99      1.00      1.00       150
shunil_gongopaddhay       0.99      0.99      0.9

**Result analysis and performance comparision**

From this work it  is seen that using 300 dimension, SVM has acchived 95% accuracy score and Logistic Regression has achieved 94% accuracy. On the other hand using 3000 dimension vector SVM has achieved 98% accuracy score and Logistic Regression has chieved 97% accuracy score which is higher compared to 300 dimension vector.

